In [1]:
import mechanize
import logging
from IPython.core.display import HTML
import sys
from bs4 import BeautifulSoup
reload(sys)
sys.setdefaultencoding('utf-8')

logger = logging.getLogger("mechanize")
logger.addHandler(logging.StreamHandler(sys.stdout))
logger.setLevel(logging.DEBUG)

In [2]:
br = mechanize.Browser()
br.set_debug_http(True)
#br.set_debug_responses(True)
br.set_debug_redirects(True)
br.set_handle_robots(False)
#br.set_handle_referer(False)
#br.set_handle_refresh(False)
br.addheaders = [('User-agent', 'Firefox')]
key = "Ingeniero de sistemas"
br.open("https://aptitus.com/buscar-trabajo-en-peru?q="+key.replace(' ', '-'))
# https://aptitus.com/buscar-trabajo-en-peru/de-tecnologia-sistemas-y-telecomunicaciones
#
print br.geturl()
search_url=br.geturl()

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:661)>

In [16]:
jobslinks=[]
temp_pagination = [l for l in br.links() if l.text.isdigit()][-1]
for d in br.links():
    if d.text.isdigit():
        br.follow_link(d)
        jobslinks+=[l for l in br.links() if l.url.startswith("https://aptitus.com/ofertas-de-trabajo/")]
#HTML(br.response().read())
len(jobslinks)

200

In [88]:
def recur():
    global jobslinks
    global temp_pagination
    print "initial",len(jobslinks)
    temp = [l for l in br.links() if l.text.isdigit()]
    if temp:
        links = [i for i in temp if int(i.text)>int(temp_pagination.text)]
        temp_pagination = temp[-1]
        print "links", len(links)
        for link in links:
            br.follow_link(link)
            jobslinks+=[l for l in br.links() if l.url.startswith("https://aptitus.com/ofertas-de-trabajo/")]
        if links:
            recur()
    print len(jobslinks)

print "##########i"
print br.geturl()
recur()
print "##########f"

In [ ]:
#HTML(br.response().read())

In [87]:
my_array=[]
for j in jobslinks:
    br.follow_link(j)
    print j.url
    soup = BeautifulSoup(br.response().read(), "html.parser")
    if not soup.body.find(text='Este aviso ha finalizado, puedes ver otros avisos similares'):
        my_array.append(
            dict(
                url=j.url,
                company=soup.select('a.b-job-detail_subtitle')[0].string,
                title=soup.select('h1.b-job-detail_title')[0].string,
                location=soup.body.find(text='Ubicación').parent.next_sibling.string,
                description=soup.select('.b-job-info')[0].get_text(),
                html=str(soup.select('.b-job-info')[0]),
                date=soup.body.find(text='Publicado').parent.next_sibling.string,
                area =soup.body.find(text='Área').parent.next_sibling.string
            )
        )

In [53]:
#[i.url for i in jobslinks]

In [89]:
len(my_array)

546

In [90]:
import json
with open('aptitus', 'w') as fout:
    json.dump(my_array, fout)